In [1]:
import pandas as pd


In [2]:
import SciServer.CasJobs as CasJobs # query with CasJobs, the primary database for the SDSS
import SciServer.SkyServer as SkyServer # show individual objects through SkyServer
import SciServer.SciDrive
import warnings
warnings.filterwarnings('ignore')


In [3]:
# df0 = pd.read_csv('new_set1000.csv')
df0 = pd.read_csv('clean-flags.csv', skiprows = 1)


In [4]:
# flags = df0['Column1']
flags = df0['flag_text']
tags = ['GALAXY']*len(flags)
df0['tags'] = tags


In [5]:
df0.loc[0, 'tags']

'GALAXY'

In [6]:
# list_lists = []
# for x in flags:
#     list_lists.append(x.split(' '))
    
# # set.intersection(*[set(list) for list in list_lists])
# list_lists

In [7]:
# set.intersection(*map(set, list_lists))

In [8]:
reject_csv = pd.read_csv('Reject Stars - twoColumn.csv')
reject_csv

,Star,objID,ra,dec,Flags,notes,Galaxy,objID.1,ra.1,dec.1,Flags.1,notes.1
0,NaN,1237661121850901163,48.999980,41.287510,DEBLEND_DEGENERATE BAD_MOVING_FIT_CHILD PEAKS_...,clearly two stars,NaN,1237661083199275572,49.693181,41.031127,DEBLENDED_AT_EDGE STATIONARY BINNED1 INTERP CO...,https://skyserver.sdss.org/dr16/en/tools/explo...
1,NaN,1237661083199078716,49.238530,41.356745,DEBLENDED_AT_EDGE STATIONARY BINNED1 SUBTRACTE...,NaN,NaN,1237670960021111926,49.410488,41.011425,DEBLENDED_AT_EDGE STATIONARY BINNED1 INTERP CO...,https://skyserver.sdss.org/dr16/en/tools/explo...
2,NaN,1237661122387837316,49.303027,41.483379,TOO_FEW_GOOD_DETECTIONS PSF_FLUX_INTERP DEBLEN...,artifact from bright star nearby https://skyse...,NaN,1237661083199145348,49.334788,41.258372,PSF_FLUX_INTERP DEBLENDED_AT_EDGE STATIONARY B...,https://skyserver.sdss.org/dr16/en/tools/explo...
3,NaN,1237661059574203088,49.670447,41.737503,DEBLENDED_AT_EDGE BAD_MOVING_FIT MOVED BINNED1...,NaN,NaN,1237661122387903518,49.518370,41.369743,PSF_FLUX_INTERP DEBLENDED_AT_EDGE INTERP_CENTE...,https://skyserver.sdss.org/dr16/en/tools/explo...
4,NaN,1237661060111007994,49.709298,42.151482,PSF_FLUX_INTERP DEBLENDED_AT_EDGE STATIONARY B...,NaN,NaN,1237661059574203219,49.647607,41.667027,PSF_FLUX_INTERP DEBLENDED_AT_EDGE BAD_MOVING_F...,https://skyserver.sdss.org/dr16/en/tools/explo...
5,NaN,1237661055818465980,49.723335,41.987172,DEBLENDED_AT_EDGE STATIONARY BINNED1 SUBTRACTE...,NaN,NaN,1237661122388034195,49.745417,41.320873,DEBLENDED_AT_EDGE STATIONARY BINNED1 INTERP CO...,https://skyserver.sdss.org/dr16/en/tools/explo...
6,NaN,1237661060111008401,49.864958,42.206252,TOO_FEW_GOOD_DETECTIONS PSF_FLUX_INTERP DEBLEN...,NaN,NaN,1237661060111009021,49.760077,42.122564,DEBLENDED_AT_EDGE BINNED1 CHILD,https://skyserver.sdss.org/dr16/en/tools/explo...
7,NaN,1237670458048512961,49.942790,41.879515,DEBLENDED_AT_EDGE STATIONARY MOVED BINNED1 INT...,NaN,NaN,1237661059574268657,49.785213,41.581111,DEBLENDED_AT_EDGE STATIONARY BINNED1 INTERP CO...,https://skyserver.sdss.org/dr16/en/tools/explo...
8,NaN,1237661060111204440,50.216443,41.971605,MAYBE_CR TOO_FEW_GOOD_DETECTIONS PSF_FLUX_INTE...,NaN,NaN,1237661083199275822,49.786257,41.126914,DEBLENDED_AT_EDGE STATIONARY BAD_MOVING_FIT BI...,https://skyserver.sdss.org/dr16/en/tools/explo...
9,NaN,1237661059574530110,50.378387,41.358666,MAYBE_CR DEBLEND_NOPEAK DEBLENDED_AT_EDGE BAD_...,NaN,NaN,1237661122388099272,49.904114,41.171023,DEBLENDED_AT_EDGE STATIONARY BINNED1 SATURATED...,https://skyserver.sdss.org/dr16/en/tools/explo...


In [9]:


starFlags = reject_csv['Flags']
starID = reject_csv['objID']

galFlags = reject_csv['Flags.1']
galID = reject_csv['objID.1']

starDf = reject_csv[['objID', 'Flags']].copy()
galDf = reject_csv[['objID.1', 'Flags.1']].copy()

In [10]:
# df_melt = pd.melt(starDf)

# df_melt
unique_star_flags = set()

for objflags in (starFlags.to_list()):
    objflags = objflags.split(' ')
    for flag in objflags:
        if flag not in unique_star_flags:
            unique_star_flags.add(flag)
unique_star_flags

{'BAD_MOVING_FIT',
 'BAD_MOVING_FIT_CHILD',
 'BINNED1',
 'BLENDED',
 'CHILD',
 'COSMIC_RAY',
 'DEBLENDED_AS_PSF',
 'DEBLENDED_AT_EDGE',
 'DEBLEND_DEGENERATE',
 'DEBLEND_NOPEAK',
 'DEBLEND_PRUNED',
 'EDGE',
 'INTERP',
 'INTERP_CENTER',
 'MANYPETRO',
 'MAYBE_CR',
 'MOVED',
 'NODEBLEND',
 'NODEBLEND_MOVING',
 'NOTCHECKED',
 'PEAKS_TOO_CLOSE',
 'PSF_FLUX_INTERP',
 'SATURATED',
 'SATUR_CENTER',
 'STATIONARY',
 'SUBTRACTED',
 'TOO_FEW_GOOD_DETECTIONS'}

In [11]:
# starDf
galDf

,objID.1,Flags.1
0,1237661083199275572,DEBLENDED_AT_EDGE STATIONARY BINNED1 INTERP CO...
1,1237670960021111926,DEBLENDED_AT_EDGE STATIONARY BINNED1 INTERP CO...
2,1237661083199145348,PSF_FLUX_INTERP DEBLENDED_AT_EDGE STATIONARY B...
3,1237661122387903518,PSF_FLUX_INTERP DEBLENDED_AT_EDGE INTERP_CENTE...
4,1237661059574203219,PSF_FLUX_INTERP DEBLENDED_AT_EDGE BAD_MOVING_F...
5,1237661122388034195,DEBLENDED_AT_EDGE STATIONARY BINNED1 INTERP CO...
6,1237661060111009021,DEBLENDED_AT_EDGE BINNED1 CHILD
7,1237661059574268657,DEBLENDED_AT_EDGE STATIONARY BINNED1 INTERP CO...
8,1237661083199275822,DEBLENDED_AT_EDGE STATIONARY BAD_MOVING_FIT BI...
9,1237661122388099272,DEBLENDED_AT_EDGE STATIONARY BINNED1 SATURATED...


In [12]:
star_flagDf = pd.DataFrame(columns = list(unique_star_flags)) 
star_flagDf

,BAD_MOVING_FIT,SATUR_CENTER,TOO_FEW_GOOD_DETECTIONS,BINNED1,SUBTRACTED,PEAKS_TOO_CLOSE,DEBLEND_NOPEAK,PSF_FLUX_INTERP,NODEBLEND,CHILD,...,DEBLENDED_AS_PSF,INTERP_CENTER,MANYPETRO,MOVED,SATURATED,INTERP,EDGE,DEBLENDED_AT_EDGE,NODEBLEND_MOVING,NOTCHECKED


In [13]:
unique_star_flags

{'BAD_MOVING_FIT',
 'BAD_MOVING_FIT_CHILD',
 'BINNED1',
 'BLENDED',
 'CHILD',
 'COSMIC_RAY',
 'DEBLENDED_AS_PSF',
 'DEBLENDED_AT_EDGE',
 'DEBLEND_DEGENERATE',
 'DEBLEND_NOPEAK',
 'DEBLEND_PRUNED',
 'EDGE',
 'INTERP',
 'INTERP_CENTER',
 'MANYPETRO',
 'MAYBE_CR',
 'MOVED',
 'NODEBLEND',
 'NODEBLEND_MOVING',
 'NOTCHECKED',
 'PEAKS_TOO_CLOSE',
 'PSF_FLUX_INTERP',
 'SATURATED',
 'SATUR_CENTER',
 'STATIONARY',
 'SUBTRACTED',
 'TOO_FEW_GOOD_DETECTIONS'}

In [14]:
# df_melt = reject_csv.assign(flags=reject_csv['Flags'].to_list().str.split(' ')) 
for key in star_flagDf.keys():
    print(key)

BAD_MOVING_FIT
SATUR_CENTER
TOO_FEW_GOOD_DETECTIONS
BINNED1
SUBTRACTED
PEAKS_TOO_CLOSE
DEBLEND_NOPEAK
PSF_FLUX_INTERP
NODEBLEND
CHILD
COSMIC_RAY
STATIONARY
DEBLEND_DEGENERATE
BAD_MOVING_FIT_CHILD
BLENDED
MAYBE_CR
DEBLEND_PRUNED
DEBLENDED_AS_PSF
INTERP_CENTER
MANYPETRO
MOVED
SATURATED
INTERP
EDGE
DEBLENDED_AT_EDGE
NODEBLEND_MOVING
NOTCHECKED


In [15]:
key = 'Flags'
# key = 'Flags.1'
def mark_row(df_row):
    
    # maps keys to vals
    row_dict = df_row.to_dict()
    
    # marks flags with hit = 1
    for flag in row_dict[key].split(): 
        row_dict[flag] = 1
        
    # returns as list to be to be used in DataFrame.apply()
    return(row_dict.values())

In [16]:
def tag_routine(df):
    pd.options.display.float_format = '{:.0f}'.format # suppress conversion to scientific notation 
    
    rename_keys = dict(enumerate(df.keys())) # hold on to index - key pairs

    ## this can be done all in one line or chain 
    df = df.apply(mark_row, axis=1, result_type = 'expand') # apply mark_row to each obj's flag list
    df = df.fillna(0) # fill nans with 0
    df = df.rename(columns = rename_keys) # renames columns back to flags on index
    
    return(df)

In [17]:
reject_stars = pd.concat([starDf, star_flagDf])

# pd.options.display.float_format = '{:.0f}'.format # suppress conversion to scientific notation 

# rename_keys = dict(enumerate(reject_stars.keys())) # hold on to index - key pairs

# reject_stars = reject_stars.apply(mark_star, axis=1, result_type = "expand") # apply mark_star to each star's flag list 
# reject_stars = reject_stars.fillna(0) # fills nans with 0
# reject_stars = reject_stars.rename(columns = rename_keys) # renames columns back to flags on index

# # reject_stars

In [18]:
# 

In [19]:
stars_out = tag_routine(reject_stars)

In [20]:
key1 = 'Flags.1'
def mark_row1(df_row):
    
    # maps keys to vals
    row_dict = df_row.to_dict()
    
    # marks flags with hit = 1
    for flag in row_dict[key1].split(): 
        row_dict[flag] = 1
        
    # returns as list to be to be used in DataFrame.apply()
    return(row_dict.values())

def tag_routine1(df):
    pd.options.display.float_format = '{:.0f}'.format # suppress conversion to scientific notation 
    
    rename_keys = dict(enumerate(df.keys())) # hold on to index - key pairs
    
    ## this can be done all in one line or chain 
    df = df.apply(mark_row1, axis=1, result_type = 'expand') # apply mark_row to each obj's flag list
    df = df.fillna(0) # fill nans with 0
    df = df.rename(columns = rename_keys) # renames columns back to flags on index
    
    return(df)

unique_gal_flags = set()

for objflags in (galFlags.to_list()):
    objflags = objflags.split(' ')
    for flag in objflags:
        if flag not in unique_gal_flags:
            unique_gal_flags.add(flag.strip())
# unique_gal_flags

gal_flagDf = pd.DataFrame(columns = list(unique_gal_flags)) 
# gal_flagDf
example_gals = pd.concat([galDf, gal_flagDf])
gals_out = tag_routine1(example_gals)


In [21]:
# stars_out.to_csv('taggedStarFlags.csv')
# gals_out.to_csv('taggedGalFlags.csv')

# stars_out.describe()
stars_out
# gals_out

,objID,Flags,BAD_MOVING_FIT,SATUR_CENTER,TOO_FEW_GOOD_DETECTIONS,BINNED1,SUBTRACTED,PEAKS_TOO_CLOSE,DEBLEND_NOPEAK,PSF_FLUX_INTERP,...,DEBLENDED_AS_PSF,INTERP_CENTER,MANYPETRO,MOVED,SATURATED,INTERP,EDGE,DEBLENDED_AT_EDGE,NODEBLEND_MOVING,NOTCHECKED
0,1237661121850901248,DEBLEND_DEGENERATE BAD_MOVING_FIT_CHILD PEAKS_...,1,0,0,1,0,1,0,0,...,0,0,1,1,0,0,0,0,1,0
1,1237661083199078656,DEBLENDED_AT_EDGE STATIONARY BINNED1 SUBTRACTE...,0,0,0,1,1,0,0,0,...,0,0,1,0,1,1,1,1,0,1
2,1237661122387837440,TOO_FEW_GOOD_DETECTIONS PSF_FLUX_INTERP DEBLEN...,0,0,1,1,0,0,1,1,...,1,0,0,0,1,1,0,1,0,0
3,1237661059574203136,DEBLENDED_AT_EDGE BAD_MOVING_FIT MOVED BINNED1...,1,0,0,1,0,0,0,0,...,0,0,0,1,1,1,0,1,0,0
4,1237661060111008000,PSF_FLUX_INTERP DEBLENDED_AT_EDGE STATIONARY B...,0,0,0,1,0,0,0,1,...,0,0,1,0,1,1,0,1,0,0
5,1237661055818466048,DEBLENDED_AT_EDGE STATIONARY BINNED1 SUBTRACTE...,0,0,0,1,1,0,0,0,...,0,0,1,0,1,1,0,1,0,0
6,1237661060111008512,TOO_FEW_GOOD_DETECTIONS PSF_FLUX_INTERP DEBLEN...,0,1,1,1,1,0,1,1,...,0,1,0,0,1,1,0,1,0,0
7,1237670458048513024,DEBLENDED_AT_EDGE STATIONARY MOVED BINNED1 INT...,0,0,0,1,0,0,0,0,...,0,0,0,1,0,1,0,1,0,0
8,1237661060111204352,MAYBE_CR TOO_FEW_GOOD_DETECTIONS PSF_FLUX_INTE...,0,1,1,1,1,0,1,1,...,0,1,0,0,1,1,0,1,0,0
9,1237661059574530048,MAYBE_CR DEBLEND_NOPEAK DEBLENDED_AT_EDGE BAD_...,1,0,0,1,0,0,1,0,...,0,0,1,0,0,1,1,1,0,1


# After some analysis in excel: 
## Most common star flags and most common gal flags were found:

In [22]:
common_star_flags = 'CHILD, BINNED1, INTERP, PSF_FLUX_INTERP, STATIONARY, SUBTRACTED, DEBLEND_NO_PEAK, DEBLENDED_AT_EDGE, MANYPETRO, SATURATED, COSMIC_RAY'
common_star_flags = common_star_flags.split(',')

common_gal_flags = 'INTERP, STATIONARY, COSMIC_RAY, DEBLENDED_AT_EDGE, CHILD, BINNED1'
common_gal_flags = common_gal_flags.split(',')


In [23]:
common_star_flags

['CHILD',
 ' BINNED1',
 ' INTERP',
 ' PSF_FLUX_INTERP',
 ' STATIONARY',
 ' SUBTRACTED',
 ' DEBLEND_NO_PEAK',
 ' DEBLENDED_AT_EDGE',
 ' MANYPETRO',
 ' SATURATED',
 ' COSMIC_RAY']

In [24]:
common_gal_flags

['INTERP',
 ' STATIONARY',
 ' COSMIC_RAY',
 ' DEBLENDED_AT_EDGE',
 ' CHILD',
 ' BINNED1']

In [25]:
galString = 'and s.z > 0.05'
starString = 'and s.z < 0.05'

In [26]:
# lambda f string that takes in str arg
# returns either stars or galaxies based off of spectroscopic redshift 

query = lambda x : f'SELECT TOP 20 p.objID, p.ra, p.dec, \
 p.modelMag_r, \
 p.modelMag_u - p.modelMag_g as u_g, \
 p.modelMag_g - p.modelMag_z as g_z, \
 p.modelMag_g - p.modelMag_r as g_r, \
 p.modelMag_g - p.modelMag_i as g_i, \
 p.modelMag_r - p.modelMag_i as r_i, \
 p.modelMag_r - p.modelMag_z as r_z, \
 p.petroRad_r, p.flags, dbo.fPhotoFlagsN(p.flags) as flag_text, \
 s.specObjID, s.z, s.zErr, s.zWarning, s.class, s.subClass, \
 N.distance \
FROM\
 photoObj as p \
JOIN SpecObjAll s ON p.objID = s.bestObjID \
JOIN dbo.fGetNearbyObjEq(49.9467, 41.5131, 45) as N ON N.objID = p.objID \
WHERE \
 p.modelMag_r > -14.3*(p.modelMag_g - p.modelMag_z) + 37 \
 and p.modelMag_r > -14.3*(p.modelMag_g - p.modelMag_z) + 46 \
 and p.type = 3 \
 {x} \
ORDER BY distance'

In [27]:
# two extract galaxy and star flags
galDf = CasJobs.executeQuery(query(galString), "dr16")
starDf = CasJobs.executeQuery(query(starString), "dr16")
galFlags = galDf['flag_text']
starFlags = starDf['flag_text']

In [28]:
# flags are in list of list format, need to flatten 
def flatten_extend(matrix):
    flat_list = []
    for row in matrix:
        flat_list.extend(row)
    return flat_list

In [29]:
# turn flag column into list of lists
g = galFlags.to_list()
gsplit = list(map(str.split, g))

# flatten and convert to set to remove duplicates 
gal_set = set(flatten_extend(gsplit))

# same for stars
s = starFlags.to_list()
ssplit = list(map(str.split, s))
star_set = set(flatten_extend(ssplit))

In [30]:
# this should give all flags present in stars but not galaxies
stars_only = star_set - gal_set
print(stars_only)

{'DEBLENDED_AS_PSF', 'TOO_FEW_GOOD_DETECTIONS', 'MANYPETRO', 'NODEBLEND_MOVING', 'BAD_MOVING_FIT_CHILD', 'MAYBE_CR', 'PEAKS_TOO_CLOSE'}


In [31]:
# function to take in set and create string for each element
flagMaker = lambda x : f"dbo.fPhotoFlags('{x}')"
conditionalStr = f'p.flags & ('
elem = map(flagMaker, stars_only)
flagsIn = '+'.join(list(elem))

flagString = 'and ' + conditionalStr + flagsIn + ') = 0'
print(flagString)

and p.flags & (dbo.fPhotoFlags('DEBLENDED_AS_PSF')+dbo.fPhotoFlags('TOO_FEW_GOOD_DETECTIONS')+dbo.fPhotoFlags('MANYPETRO')+dbo.fPhotoFlags('NODEBLEND_MOVING')+dbo.fPhotoFlags('BAD_MOVING_FIT_CHILD')+dbo.fPhotoFlags('MAYBE_CR')+dbo.fPhotoFlags('PEAKS_TOO_CLOSE')) = 0


In [32]:
query2 = lambda flags: f'SELECT TOP 20 p.objID, p.ra, p.dec, \
 p.modelMag_r, \
 p.modelMag_u - p.modelMag_g as u_g, \
 p.modelMag_g - p.modelMag_z as g_z, \
 p.modelMag_g - p.modelMag_r as g_r, \
 p.modelMag_g - p.modelMag_i as g_i, \
 p.modelMag_r - p.modelMag_i as r_i, \
 p.modelMag_r - p.modelMag_z as r_z, \
 p.petroRad_r, p.flags, dbo.fPhotoFlagsN(p.flags) as flag_text, \
 s.specObjID, s.z, s.zErr, s.zWarning, s.class, s.subClass, \
 N.distance \
FROM\
 photoObj as p \
JOIN SpecObjAll s ON p.objID = s.bestObjID \
JOIN dbo.fGetNearbyObjEq(49.9467, 41.5131, 45) as N ON N.objID = p.objID \
WHERE \
 p.modelMag_r > -14.3*(p.modelMag_g - p.modelMag_z) + 37 \
 and p.modelMag_r > -14.3*(p.modelMag_g - p.modelMag_z) + 46 \
 and p.type = 3 \
 {flags} \
ORDER BY distance'

In [33]:
# query2(galString, flagString)
newDf = CasJobs.executeQuery(query2(flagString), "dr16")
newDf

,objID,ra,dec,modelMag_r,u_g,g_z,g_r,g_i,r_i,r_z,petroRad_r,flags,flag_text,specObjID,z,zErr,zWarning,class,subClass,distance
0,1237661059574334114,50,42,18,3,3,2,2,1,1,1,105624251793752,DEBLEND_NOPEAK DEBLENDED_AT_EDGE STATIONARY MO...,1875902951730472960,-0,0,0,STAR,M1,4
1,1237661059574399247,50,41,18,3,3,2,2,1,1,1,35253360136208,DEBLENDED_AT_EDGE STATIONARY BINNED1 INTERP CO...,1875796848858392576,0,0,0,STAR,M0,4
2,1237661059574399247,50,41,18,3,3,2,2,1,1,1,35253360136208,DEBLENDED_AT_EDGE STATIONARY BINNED1 INTERP CO...,1874672323089426432,0,0,0,STAR,M0,4
3,1237661122388033863,50,41,17,2,2,1,2,1,1,18,35253360132368,DEBLENDED_AT_EDGE STATIONARY BINNED1 INTERP NO...,1874671498455705600,0,0,0,GALAXY,,7
4,1237661055281857482,50,41,18,2,2,1,2,1,1,4,35259802583056,DEBLENDED_AT_EDGE STATIONARY DEBLENDED_AS_MOVI...,1874672872845240320,0,0,0,GALAXY,,11
5,1237661055281725850,50,41,13,2,2,2,2,1,1,16,35219001708560,DEBLENDED_AT_EDGE BAD_MOVING_FIT BINNED1 SUBTR...,1875813891288623104,0,0,0,GALAXY,,12
6,1237661059574269120,50,42,18,2,3,1,2,1,2,1,105589892055056,DEBLEND_NOPEAK DEBLENDED_AT_EDGE BAD_MOVING_FI...,1875883985154893824,-0,0,0,STAR,K7,13
7,1237661059574269035,50,42,17,3,2,1,2,1,1,1,68987912192,STATIONARY BINNED1,1875882610765359104,-0,0,0,STAR,K7,15
8,1237661059574269035,50,42,17,3,2,1,2,1,1,1,68987912192,STATIONARY BINNED1,1874758909630113792,-0,0,0,STAR,K7,15
9,1237661055818663393,50,42,18,2,2,1,2,1,1,3,35259802587152,DEBLENDED_AT_EDGE STATIONARY DEBLENDED_AS_MOVI...,1875901027585124352,0,0,0,GALAXY,,15


In [34]:
query3 = '''CREATE TABLE #upload ( up_id int, up_files varchar(32), up_name varchar(32), up_labels varchar(32), up_reds varchar(32), up_ra float, up_dec float ) 
INSERT INTO #upload values ( 1, 'sdss_ra=49.5885_dec=41.6896.png', 'PCC-2087', '0', '2.39', 49.5885, 41.6896),( 2, 'sdss_ra=49.4776_dec=41.384.png', 'PCC-1378', '0', '6.325', 49.4776, 41.384),( 3, 'sdss_ra=50.004_dec=41.341.png', 'PCC-5423', '0', '5.62', 50.004, 41.341),( 4, 'sdss_ra=49.4142_dec=41.3882.png', 'PCC-0989', '0', '1.6325', 49.4142, 41.3882),( 5, 'sdss_ra=50.0018_dec=41.6806.png', 'PCC-5408', '0', '1.8725', 50.0018, 41.6806),( 6, 'sdss_ra=49.6743_dec=41.3287.png', 'PCC-2818', '0', '0.04', 49.6743, 41.3287),( 7, 'sdss_ra=49.4266_dec=41.3159.png', 'PCC-1064', '0', '1.8225', 49.4266, 41.3159),( 8, 'sdss_ra=49.3303_dec=41.5381.png', 'PCC-0494', '0', '2.5825', 49.3303, 41.5381),( 9, 'sdss_ra=49.8286_dec=41.2883.png', 'PCC-4039', '0', '0.1675', 49.8286, 41.2883),( 10, 'sdss_ra=49.3925_dec=41.5647.png', 'PCC-0870', '0', '1.265', 49.3925, 41.5647),( 11, 'sdss_ra=49.4784_dec=41.2846.png', 'PCC-1388', '0', '3.62', 49.4784, 41.2846),( 12, 'sdss_ra=49.5392_dec=41.4577.png', 'PCC-1782', '0', '2.515', 49.5392, 41.4577),( 13, 'sdss_ra=49.7004_dec=41.5288.png', 'PCC-3025', '0', '1.695', 49.7004, 41.5288),( 14, 'sdss_ra=49.7521_dec=41.666.png', 'PCC-3450', '0', '1.74', 49.7521, 41.666),( 15, 'sdss_ra=49.7463_dec=41.6377.png', 'PCC-3408', '0', '5.7625', 49.7463, 41.6377),( 16, 'sdss_ra=49.7314_dec=41.5563.png', 'PCC-3286', '0', '17.815', 49.7314, 41.5563),( 17, 'sdss_ra=49.5063_dec=41.5123.png', 'PCC-1565', '0', '6.8275', 49.5063, 41.5123),( 18, 'sdss_ra=49.5319_dec=41.5225.png', 'PCC-1742', '0', '2.4475', 49.5319, 41.5225),( 19, 'sdss_ra=49.5132_dec=41.3783.png', 'PCC-1617', '0', '2.0275', 49.5132, 41.3783),( 20, 'sdss_ra=49.4477_dec=41.575.png', 'PCC-1191', '0', '1.6075', 49.4477, 41.575),( 21, 'sdss_ra=49.2565_dec=41.4396.png', 'PCC-0057', '0', '3.51', 49.2565, 41.4396),( 22, 'sdss_ra=49.3508_dec=41.3929.png', 'PCC-0626', '0', '19.445', 49.3508, 41.3929),( 23, 'sdss_ra=49.5144_dec=41.6052.png', 'PCC-1623', '0', '17.38', 49.5144, 41.6052),( 24, 'sdss_ra=49.3878_dec=41.5814.png', 'PCC-0826', '0', '0.705', 49.3878, 41.5814),( 25, 'sdss_ra=49.7118_dec=41.7236.png', 'PCC-3109', '0', '1.48', 49.7118, 41.7236),( 26, 'sdss_ra=49.7624_dec=41.308.png', 'PCC-3531', '0', '4.0975', 49.7624, 41.308),( 27, 'sdss_ra=49.8839_dec=41.7175.png', 'PCC-4515', '0', '6.82', 49.8839, 41.7175),( 28, 'sdss_ra=49.7006_dec=41.4398.png', 'PCC-3028', '0', '9.215', 49.7006, 41.4398),( 29, 'sdss_ra=49.6028_dec=41.5367.png', 'PCC-2195', '0', '8.365', 49.6028, 41.5367),( 30, 'sdss_ra=49.2466_dec=41.4451.png', 'PCC-0026', '0', '0.445', 49.2466, 41.4451),( 31, 'sdss_ra=49.6212_dec=41.7373.png', 'PCC-2365', '0', '2.2225', 49.6212, 41.7373),( 32, 'sdss_ra=49.407_dec=41.3729.png', 'PCC-0954', '0', '2.32', 49.407, 41.3729),( 33, 'sdss_ra=49.3329_dec=41.4999.png', 'PCC-0514', '0', '0.92', 49.3329, 41.4999),( 34, 'sdss_ra=49.2744_dec=41.5706.png', 'PCC-0160', '0', '1.22', 49.2744, 41.5706),( 35, 'sdss_ra=49.4898_dec=41.5315.png', 'PCC-1467', '0', '13.855', 49.4898, 41.5315),( 36, 'sdss_ra=49.4525_dec=41.5851.png', 'PCC-1222', '0', '1.9275', 49.4525, 41.5851),( 37, 'sdss_ra=49.3368_dec=41.4033.png', 'PCC-0538', '0', '13.4775', 49.3368, 41.4033),( 38, 'sdss_ra=49.5331_dec=41.4152.png', 'PCC-1749', '0', '2.0575', 49.5331, 41.4152),( 39, 'sdss_ra=49.5756_dec=41.4865.png', 'PCC-1992', '0', '2.2225', 49.5756, 41.4865),( 40, 'sdss_ra=50.0051_dec=41.6557.png', 'PCC-5428', '0', '1.8225', 50.0051, 41.6557),( 41, 'sdss_ra=49.7564_dec=41.2183.png', 'PCC-3477', '0', '6.9975', 49.7564, 41.2183),( 42, 'sdss_ra=49.6283_dec=41.2297.png', 'PCC-2441', '0', '13.07', 49.6283, 41.2297),( 43, 'sdss_ra=49.3541_dec=41.4703.png', 'PCC-0641', '0', '10.3425', 49.3541, 41.4703),( 44, 'sdss_ra=49.3081_dec=41.4699.png', 'PCC-0365', '0', '6.7025', 49.3081, 41.4699),( 45, 'sdss_ra=49.3881_dec=41.3949.png', 'PCC-0828', '0', '13.7625', 49.3881, 41.3949),( 46, 'sdss_ra=49.6684_dec=41.258.png', 'PCC-2760', '0', '4.1625', 49.6684, 41.258),( 47, 'sdss_ra=49.5907_dec=41.6753.png', 'PCC-2113', '0', '8.5275', 49.5907, 41.6753),( 48, 'sdss_ra=49.5877_dec=41.3917.png', 'PCC-2079', '0', '3.29', 49.5877, 41.3917),( 49, 'sdss_ra=49.4508_dec=41.5244.png', 'PCC-1208', '0', '0.745', 49.4508, 41.5244),( 50, 'sdss_ra=49.6184_dec=41.4546.png', 'PCC-2341', '0', '1.7125', 49.6184, 41.4546),( 51, 'sdss_ra=49.3497_dec=41.2962.png', 'PCC-0619', '0', '1.265', 49.3497, 41.2962),( 52, 'sdss_ra=49.9814_dec=41.4202.png', 'PCC-5240', '0', '1.585', 49.9814, 41.4202),( 53, 'sdss_ra=49.5539_dec=41.502.png', 'PCC-1867', '0', '10.2025', 49.5539, 41.502),( 54, 'sdss_ra=49.5094_dec=41.5135.png', 'PCC-1592', '0', '11.1625', 49.5094, 41.5135),( 55, 'sdss_ra=49.6309_dec=41.2656.png', 'PCC-2462', '0', '7.75', 49.6309, 41.2656),( 56, 'sdss_ra=49.4709_dec=41.3465.png', 'PCC-1336', '0', '0.0125', 49.4709, 41.3465),( 57, 'sdss_ra=49.2475_dec=41.3259.png', 'PCC-0027', '0', '1.7425', 49.2475, 41.3259),( 58, 'sdss_ra=49.623_dec=41.7693.png', 'PCC-2383', '0', '2.5125', 49.623, 41.7693),( 59, 'sdss_ra=49.9366_dec=41.7691.png', 'PCC-4910', '0', '2.935', 49.9366, 41.7691),( 60, 'sdss_ra=49.5669_dec=41.3688.png', 'PCC-1940', '0', '1.5825', 49.5669, 41.3688),( 61, 'sdss_ra=49.4854_dec=41.4154.png', 'PCC-1433', '0', '3.59', 49.4854, 41.4154),( 62, 'sdss_ra=49.4978_dec=41.6188.png', 'PCC-1515', '0', '3.41', 49.4978, 41.6188),( 63, 'sdss_ra=49.8832_dec=41.3043.png', 'PCC-4508', '0', '9.5525', 49.8832, 41.3043),( 64, 'sdss_ra=49.9842_dec=41.2138.png', 'PCC-5265', '0', '0.8025', 49.9842, 41.2138),( 65, 'sdss_ra=49.4511_dec=41.3201.png', 'PCC-1212', '0', '0.665', 49.4511, 41.3201),( 66, 'sdss_ra=49.7175_dec=41.6788.png', 'PCC-3159', '0', '12.7875', 49.7175, 41.6788),( 67, 'sdss_ra=49.7208_dec=41.5487.png', 'PCC-3190', '0', '42.3225', 49.7208, 41.5487),( 68, 'sdss_ra=49.9435_dec=41.6033.png', 'PCC-4967', '0', '18.6625', 49.9435, 41.6033),( 69, 'sdss_ra=49.8546_dec=41.2312.png', 'PCC-4260', '0', '4.5675', 49.8546, 41.2312),( 70, 'sdss_ra=49.5411_dec=41.5389.png', 'PCC-1795', '0', '6.025', 49.5411, 41.5389),( 71, 'sdss_ra=49.5078_dec=41.4149.png', 'PCC-1578', '0', '0.3725', 49.5078, 41.4149),( 72, 'sdss_ra=49.4815_dec=41.434.png', 'PCC-1406', '0', '0.8525', 49.4815, 41.434),( 73, 'sdss_ra=49.5511_dec=41.4042.png', 'PCC-1849', '0', '0.89', 49.5511, 41.4042),( 74, 'sdss_ra=49.9437_dec=41.3786.png', 'PCC-4969', '0', '7.01', 49.9437, 41.3786),( 75, 'sdss_ra=49.5475_dec=41.5225.png', 'PCC-1827', '0', '10.325', 49.5475, 41.5225),( 76, 'sdss_ra=49.8615_dec=41.4943.png', 'PCC-4319', '0', '23.2275', 49.8615, 41.4943),( 77, 'sdss_ra=49.6876_dec=41.226.png', 'PCC-2928', '0', '4.625', 49.6876, 41.226),( 78, 'sdss_ra=49.2669_dec=41.5264.png', 'PCC-0110', '0', '2.055', 49.2669, 41.5264),( 79, 'sdss_ra=49.3766_dec=41.5364.png', 'PCC-0774', '0', '0.0975', 49.3766, 41.5364),( 80, 'sdss_ra=49.5082_dec=41.4069.png', 'PCC-1581', '0', '2.4', 49.5082, 41.4069),( 81, 'sdss_ra=49.9564_dec=41.3138.png', 'PCC-5087', '0', '31.96', 49.9564, 41.3138),( 82, 'sdss_ra=49.7705_dec=41.3615.png', 'PCC-3605', '0', '7.835', 49.7705, 41.3615),( 83, 'sdss_ra=49.3431_dec=41.5987.png', 'PCC-0571', '0', '1.5575', 49.3431, 41.5987),( 84, 'sdss_ra=49.5268_dec=41.3295.png', 'PCC-1707', '0', '6.0975', 49.5268, 41.3295),( 85, 'sdss_ra=49.6744_dec=41.3395.png', 'PCC-2819', '0', '1.1975', 49.6744, 41.3395),( 86, 'sdss_ra=49.9834_dec=41.4553.png', 'PCC-5257', '0', '5.9725', 49.9834, 41.4553),( 87, 'sdss_ra=49.5811_dec=41.3339.png', 'PCC-2031', '0', '0.6825', 49.5811, 41.3339),( 88, 'sdss_ra=49.8625_dec=41.6234.png', 'PCC-4328', '0', '4.6825', 49.8625, 41.6234),( 89, 'sdss_ra=49.7875_dec=41.5292.png', 'PCC-3700', '0', '11.1775', 49.7875, 41.5292),( 90, 'sdss_ra=49.9985_dec=41.3856.png', 'PCC-5387', '0', '3.3975', 49.9985, 41.3856),( 91, 'sdss_ra=49.6469_dec=41.4505.png', 'PCC-2605', '0', '3.2575', 49.6469, 41.4505),( 92, 'sdss_ra=49.242_dec=41.4454.png', 'PCC-0011', '0', '0.24', 49.242, 41.4454),( 93, 'sdss_ra=49.4487_dec=41.3384.png', 'PCC-1198', '0', '1.0375', 49.4487, 41.3384),( 94, 'sdss_ra=49.9154_dec=41.2302.png', 'PCC-4755', '0', '3.9775', 49.9154, 41.2302),( 95, 'sdss_ra=49.5705_dec=41.4591.png', 'PCC-1967', '0', '4.16', 49.5705, 41.4591),( 96, 'sdss_ra=49.3855_dec=41.3037.png', 'PCC-0817', '0', '10.1125', 49.3855, 41.3037),( 97, 'sdss_ra=49.3893_dec=41.5721.png', 'PCC-0837', '0', '0.4225', 49.3893, 41.5721),( 98, 'sdss_ra=49.6164_dec=41.4551.png', 'PCC-2329', '0', '1.3825', 49.6164, 41.4551),( 99, 'sdss_ra=49.36_dec=41.3234.png', 'PCC-0675', '0', '0.555', 49.36, 41.3234),( 100, 'sdss_ra=49.985_dec=41.3319.png', 'PCC-5271', '0', '4.315', 49.985, 41.3319),( 101, 'sdss_ra=49.6116_dec=41.197.png', 'PCC-2283', '0', '1.5825', 49.6116, 41.197),( 102, 'sdss_ra=49.3594_dec=41.3355.png', 'PCC-0670', '0', '0.485', 49.3594, 41.3355),( 103, 'sdss_ra=49.5006_dec=41.3793.png', 'PCC-1530', '0', '0.575', 49.5006, 41.3793),( 104, 'sdss_ra=49.6822_dec=41.2678.png', 'PCC-2878', '0', '2.2375', 49.6822, 41.2678),( 105, 'sdss_ra=50.0028_dec=41.3384.png', 'PCC-5417', '0', '3.2525', 50.0028, 41.3384),( 106, 'sdss_ra=49.3464_dec=41.4658.png', 'PCC-0592', '0', '7.6125', 49.3464, 41.4658),( 107, 'sdss_ra=49.5247_dec=41.4305.png', 'PCC-1693', '0', '0.4275', 49.5247, 41.4305),( 108, 'sdss_ra=49.7345_dec=41.6346.png', 'PCC-3313', '0', '4.3675', 49.7345, 41.6346),( 109, 'sdss_ra=49.5236_dec=41.5182.png', 'PCC-1689', '0', '1.99', 49.5236, 41.5182),( 110, 'sdss_ra=49.4158_dec=41.3219.png', 'PCC-0997', '0', '3.605', 49.4158, 41.3219),( 111, 'sdss_ra=49.2411_dec=41.4991.png', 'PCC-0008', '0', '4.1925', 49.2411, 41.4991),( 112, 'sdss_ra=49.2758_dec=41.5415.png', 'PCC-0168', '0', '5.9025', 49.2758, 41.5415),( 113, 'sdss_ra=49.5591_dec=41.5027.png', 'PCC-1896', '0', '12.3575', 49.5591, 41.5027),( 114, 'sdss_ra=49.6769_dec=41.3202.png', 'PCC-2835', '0', '5.01', 49.6769, 41.3202),( 115, 'sdss_ra=49.8577_dec=41.3264.png', 'PCC-4286', '0', '3.9675', 49.8577, 41.3264),( 116, 'sdss_ra=49.2516_dec=41.3224.png', 'PCC-0040', '1', '4.3075', 49.2516, 41.3224),( 117, 'sdss_ra=49.2835_dec=41.3141.png', 'PCC-0219', '1', '3.31', 49.2835, 41.3141),( 118, 'sdss_ra=49.2874_dec=41.4252.png', 'PCC-0246', '1', '3.59', 49.2874, 41.4252),( 119, 'sdss_ra=49.3008_dec=41.3811.png', 'PCC-0314', '1', '11.765', 49.3008, 41.3811),( 120, 'sdss_ra=49.3048_dec=41.3333.png', 'PCC-0349', '1', '12.4125', 49.3048, 41.3333),( 121, 'sdss_ra=49.3063_dec=41.2975.png', 'PCC-0358', '1', '3.5275', 49.3063, 41.2975),( 122, 'sdss_ra=49.3067_dec=41.4355.png', 'PCC-0360', '1', '0.7175', 49.3067, 41.4355),( 123, 'sdss_ra=49.3237_dec=41.4215.png', 'PCC-0462', '1', '7.3625', 49.3237, 41.4215),( 124, 'sdss_ra=49.3256_dec=41.4929.png', 'PCC-0469', '1', '1.3375', 49.3256, 41.4929),( 125, 'sdss_ra=49.3476_dec=41.47.png', 'PCC-0602', '1', '13.6375', 49.3476, 41.47),( 126, 'sdss_ra=49.3663_dec=41.3535.png', 'PCC-0708', '1', '9.7675', 49.3663, 41.3535),( 127, 'sdss_ra=49.387_dec=41.297.png', 'PCC-0823', '1', '1.22', 49.387, 41.297),( 128, 'sdss_ra=49.4082_dec=41.3954.png', 'PCC-0958', '1', '10.75', 49.4082, 41.3954),( 129, 'sdss_ra=49.4134_dec=41.5176.png', 'PCC-0985', '1', '1.3875', 49.4134, 41.5176),( 130, 'sdss_ra=49.4149_dec=41.5612.png', 'PCC-0993', '1', '1.1775', 49.4149, 41.5612),( 131, 'sdss_ra=49.4273_dec=41.3231.png', 'PCC-1067', '1', '12.985', 49.4273, 41.3231),( 132, 'sdss_ra=49.4284_dec=41.5999.png', 'PCC-1078', '1', '8.705', 49.4284, 41.5999),( 133, 'sdss_ra=49.432_dec=41.3634.png', 'PCC-1101', '1', '8.7475', 49.432, 41.3634),( 134, 'sdss_ra=49.4519_dec=41.4084.png', 'PCC-1220', '1', '2.6575', 49.4519, 41.4084),( 135, 'sdss_ra=49.4615_dec=41.522.png', 'PCC-1282', '1', '0.8025', 49.4615, 41.522),( 136, 'sdss_ra=49.5028_dec=41.2991.png', 'PCC-1543', '1', '4.92', 49.5028, 41.2991),( 137, 'sdss_ra=49.5078_dec=41.5578.png', 'PCC-1577', '1', '1.0875', 49.5078, 41.5578),( 138, 'sdss_ra=49.5126_dec=41.5128.png', 'PCC-1613', '1', '8.615', 49.5126, 41.5128),( 139, 'sdss_ra=49.5154_dec=41.3231.png', 'PCC-1628', '1', '9.28', 49.5154, 41.3231),( 140, 'sdss_ra=49.5184_dec=41.3698.png', 'PCC-1655', '1', '5.9375', 49.5184, 41.3698),( 141, 'sdss_ra=49.5231_dec=41.4355.png', 'PCC-1684', '1', '3.385', 49.5231, 41.4355),( 142, 'sdss_ra=49.5244_dec=41.5814.png', 'PCC-1691', '1', '6.1025', 49.5244, 41.5814),( 143, 'sdss_ra=49.5308_dec=41.5539.png', 'PCC-1736', '1', '3.3825', 49.5308, 41.5539),( 144, 'sdss_ra=49.5486_dec=41.6169.png', 'PCC-1833', '1', '7.3675', 49.5486, 41.6169),( 145, 'sdss_ra=49.5502_dec=41.4233.png', 'PCC-1842', '1', '15.6625', 49.5502, 41.4233),( 146, 'sdss_ra=49.557_dec=41.4616.png', 'PCC-1884', '1', '2.215', 49.557, 41.4616),( 147, 'sdss_ra=49.5667_dec=41.2776.png', 'PCC-1937', '1', '9.0875', 49.5667, 41.2776),( 148, 'sdss_ra=49.568_dec=41.5799.png', 'PCC-1947', '1', '1.21', 49.568, 41.5799),( 149, 'sdss_ra=49.5741_dec=41.5031.png', 'PCC-1984', '1', '13.1275', 49.5741, 41.5031),( 150, 'sdss_ra=49.5791_dec=41.6937.png', 'PCC-2021', '1', '2.435', 49.5791, 41.6937),( 151, 'sdss_ra=49.582_dec=41.2943.png', 'PCC-2039', '1', '0.8525', 49.582, 41.2943),( 152, 'sdss_ra=49.5958_dec=41.2689.png', 'PCC-2146', '1', '6.9075', 49.5958, 41.2689),( 153, 'sdss_ra=49.5992_dec=41.6442.png', 'PCC-2172', '1', '29.7875', 49.5992, 41.6442),( 154, 'sdss_ra=49.6048_dec=41.5212.png', 'PCC-2214', '1', '1.2525', 49.6048, 41.5212),( 155, 'sdss_ra=49.6078_dec=41.5452.png', 'PCC-2241', '1', '5.425', 49.6078, 41.5452),( 156, 'sdss_ra=49.6152_dec=41.3213.png', 'PCC-2325', '1', '5.2975', 49.6152, 41.3213),( 157, 'sdss_ra=49.6169_dec=41.7091.png', 'PCC-2332', '1', '20.26', 49.6169, 41.7091),( 158, 'sdss_ra=49.6181_dec=41.286.png', 'PCC-2339', '1', '2.9925', 49.6181, 41.286),( 159, 'sdss_ra=49.6241_dec=41.3027.png', 'PCC-2393', '1', '8.96', 49.6241, 41.3027),( 160, 'sdss_ra=49.6382_dec=41.2977.png', 'PCC-2531', '1', '0.6525', 49.6382, 41.2977),( 161, 'sdss_ra=49.6396_dec=41.7522.png', 'PCC-2546', '1', '9.555', 49.6396, 41.7522),( 162, 'sdss_ra=49.6431_dec=41.4925.png', 'PCC-2571', '1', '3.9975', 49.6431, 41.4925),( 163, 'sdss_ra=49.6673_dec=41.4173.png', 'PCC-2754', '1', '2.19', 49.6673, 41.4173),( 164, 'sdss_ra=49.6693_dec=41.2867.png', 'PCC-2772', '1', '2.3975', 49.6693, 41.2867),( 165, 'sdss_ra=49.6694_dec=41.6242.png', 'PCC-2774', '1', '23.785', 49.6694, 41.6242),( 166, 'sdss_ra=49.6869_dec=41.6338.png', 'PCC-2918', '1', '2.355', 49.6869, 41.6338),( 167, 'sdss_ra=49.6925_dec=41.4049.png', 'PCC-2959', '1', '1.8125', 49.6925, 41.4049),( 168, 'sdss_ra=49.6929_dec=41.6028.png', 'PCC-2961', '1', '4.5675', 49.6929, 41.6028),( 169, 'sdss_ra=49.6991_dec=41.748.png', 'PCC-3010', '1', '6.1725', 49.6991, 41.748),( 170, 'sdss_ra=49.7077_dec=41.5067.png', 'PCC-3076', '1', '12.3', 49.7077, 41.5067),( 171, 'sdss_ra=49.712_dec=41.6398.png', 'PCC-3116', '1', '9.2625', 49.712, 41.6398),( 172, 'sdss_ra=49.7177_dec=41.2676.png', 'PCC-3164', '1', '8.89', 49.7177, 41.2676),( 173, 'sdss_ra=49.7227_dec=41.3095.png', 'PCC-3205', '1', '16.1025', 49.7227, 41.3095),( 174, 'sdss_ra=49.727_dec=41.2225.png', 'PCC-3244', '1', '5.9825', 49.727, 41.2225),( 175, 'sdss_ra=49.7333_dec=41.5787.png', 'PCC-3303', '1', '4.17', 49.7333, 41.5787),( 176, 'sdss_ra=49.7334_dec=41.6862.png', 'PCC-3305', '1', '6.01', 49.7334, 41.6862),( 177, 'sdss_ra=49.735_dec=41.2599.png', 'PCC-3316', '1', '2.2425', 49.735, 41.2599),( 178, 'sdss_ra=49.7397_dec=41.359.png', 'PCC-3361', '1', '6.2725', 49.7397, 41.359),( 179, 'sdss_ra=49.7405_dec=41.4161.png', 'PCC-3366', '1', '33.7925', 49.7405, 41.4161),( 180, 'sdss_ra=49.7415_dec=41.6351.png', 'PCC-3379', '1', '8.6625', 49.7415, 41.6351),( 181, 'sdss_ra=49.7454_dec=41.3209.png', 'PCC-3401', '1', '8.215', 49.7454, 41.3209),( 182, 'sdss_ra=49.7484_dec=41.4446.png', 'PCC-3418', '1', '3.915', 49.7484, 41.4446),( 183, 'sdss_ra=49.7516_dec=41.484.png', 'PCC-3444', '1', '10.1325', 49.7516, 41.484),( 184, 'sdss_ra=49.7624_dec=41.5043.png', 'PCC-3529', '1', '7.785', 49.7624, 41.5043),( 185, 'sdss_ra=49.7817_dec=41.2523.png', 'PCC-3661', '1', '5.35', 49.7817, 41.2523),( 186, 'sdss_ra=49.7852_dec=41.5811.png', 'PCC-3687', '1', '11.5075', 49.7852, 41.5811),( 187, 'sdss_ra=49.7895_dec=41.7047.png', 'PCC-3723', '1', '1.0425', 49.7895, 41.7047),( 188, 'sdss_ra=49.7926_dec=41.6781.png', 'PCC-3742', '1', '8.12', 49.7926, 41.6781),( 189, 'sdss_ra=49.7935_dec=41.4936.png', 'PCC-3749', '1', '15.975', 49.7935, 41.4936),( 190, 'sdss_ra=49.7995_dec=41.7588.png', 'PCC-3799', '1', '1.625', 49.7995, 41.7588),( 191, 'sdss_ra=49.8124_dec=41.6778.png', 'PCC-3919', '1', '1.6575', 49.8124, 41.6778),( 192, 'sdss_ra=49.8126_dec=41.2313.png', 'PCC-3921', '1', '7.695', 49.8126, 41.2313),( 193, 'sdss_ra=49.8178_dec=41.7537.png', 'PCC-3963', '1', '4.4325', 49.8178, 41.7537),( 194, 'sdss_ra=49.8495_dec=41.6924.png', 'PCC-4208', '1', '10.34', 49.8495, 41.6924),( 195, 'sdss_ra=49.8501_dec=41.5332.png', 'PCC-4216', '1', '31.8775', 49.8501, 41.5332),( 196, 'sdss_ra=49.8585_dec=41.2621.png', 'PCC-4297', '1', '2.2225', 49.8585, 41.2621),( 197, 'sdss_ra=49.8627_dec=41.6082.png', 'PCC-4330', '1', '7.4875', 49.8627, 41.6082),( 198, 'sdss_ra=49.8715_dec=41.7361.png', 'PCC-4402', '1', '2.66', 49.8715, 41.7361),( 199, 'sdss_ra=49.8823_dec=41.5225.png', 'PCC-4499', '1', '20.3575', 49.8823, 41.5225),( 200, 'sdss_ra=49.8902_dec=41.5536.png', 'PCC-4551', '1', '9.5975', 49.8902, 41.5536),( 201, 'sdss_ra=49.8969_dec=41.6179.png', 'PCC-4608', '1', '6.21', 49.8969, 41.6179),( 202, 'sdss_ra=49.9053_dec=41.486.png', 'PCC-4666', '1', '11.84', 49.9053, 41.486),( 203, 'sdss_ra=49.9145_dec=41.2779.png', 'PCC-4743', '1', '15.625', 49.9145, 41.2779),( 204, 'sdss_ra=49.9151_dec=41.5179.png', 'PCC-4750', '1', '17.335', 49.9151, 41.5179),( 205, 'sdss_ra=49.9175_dec=41.3292.png', 'PCC-4771', '1', '4.87', 49.9175, 41.3292),( 206, 'sdss_ra=49.9235_dec=41.4881.png', 'PCC-4811', '1', '18.5525', 49.9235, 41.4881),( 207, 'sdss_ra=49.9248_dec=41.4995.png', 'PCC-4816', '1', '30.525', 49.9248, 41.4995),( 208, 'sdss_ra=49.9317_dec=41.7131.png', 'PCC-4867', '1', '9.17', 49.9317, 41.7131),( 209, 'sdss_ra=49.9326_dec=41.457.png', 'PCC-4876', '1', '15.7675', 49.9326, 41.457),( 210, 'sdss_ra=49.936_dec=41.4465.png', 'PCC-4900', '1', '17.76', 49.936, 41.4465),( 211, 'sdss_ra=49.9449_dec=41.5293.png', 'PCC-4979', '1', '14.675', 49.9449, 41.5293),( 212, 'sdss_ra=49.9453_dec=41.3786.png', 'PCC-4981', '1', '9.1025', 49.9453, 41.3786),( 213, 'sdss_ra=49.9523_dec=41.5579.png', 'PCC-5047', '1', '28.535', 49.9523, 41.5579),( 214, 'sdss_ra=49.9538_dec=41.5834.png', 'PCC-5063', '1', '16.9225', 49.9538, 41.5834),( 215, 'sdss_ra=49.9574_dec=41.3299.png', 'PCC-5095', '1', '17.1275', 49.9574, 41.3299),( 216, 'sdss_ra=49.9575_dec=41.6693.png', 'PCC-5096', '1', '1.0675', 49.9575, 41.6693),( 217, 'sdss_ra=49.9635_dec=41.5354.png', 'PCC-5136', '1', '25.4425', 49.9635, 41.5354),( 218, 'sdss_ra=49.9636_dec=41.3097.png', 'PCC-5137', '1', '7.905', 49.9636, 41.3097),( 219, 'sdss_ra=49.9656_dec=41.3923.png', 'PCC-5147', '1', '2.3325', 49.9656, 41.3923),( 220, 'sdss_ra=49.967_dec=41.2598.png', 'PCC-5156', '1', '2.78', 49.967, 41.2598),( 221, 'sdss_ra=49.9672_dec=41.648.png', 'PCC-5157', '1', '2.3475', 49.9672, 41.648),( 222, 'sdss_ra=49.9686_dec=41.5498.png', 'PCC-5163', '1', '9.2', 49.9686, 41.5498),( 223, 'sdss_ra=49.9705_dec=41.6088.png', 'PCC-5169', '1', '6.5925', 49.9705, 41.6088),( 224, 'sdss_ra=49.9756_dec=41.3089.png', 'PCC-5196', '1', '11.9175', 49.9756, 41.3089),( 225, 'sdss_ra=49.9932_dec=41.5479.png', 'PCC-5339', '1', '28.4075', 49.9932, 41.5479),( 226, 'sdss_ra=49.9947_dec=41.75.png', 'PCC-5358', '1', '49.0425', 49.9947, 41.75),( 227, 'sdss_ra=49.9967_dec=41.3092.png', 'PCC-5374', '1', '17.355', 49.9967, 41.3092),( 228, 'sdss_ra=49.8825_dec=41.7447.png', 'PCC-4502', '1', '4.63', 49.8825, 41.7447),( 229, 'sdss_ra=49.9387_dec=41.2544.png', 'PCC-4932', '1', '5.585', 49.9387, 41.2544)
create table #x (up_id int,objID bigint)
INSERT INTO #x 
SELECT up_id, dbo.fGetNearestObjIdEq(up_ra,up_dec,0.03) as objId 
     FROM #upload WHERE dbo.fGetNearestObjIdEq(up_ra,up_dec,0.03) IS NOT NULL 
SELECT u.up_files as [files],u.up_name as [name],u.up_labels as [labels],u.up_reds as [reds], 
p.objID, 
dbo.fPhotoTypeN(p.type) as type,
p.ra, p.dec,
p.modelMag_r, 
p.modelMag_u - p.modelMag_g as u_g, 
p.modelMag_g - p.modelMag_z as g_z, 
p.modelMag_g - p.modelMag_r as g_r, 
p.modelMag_g - p.modelMag_i as g_i, 
p.modelMag_r - p.modelMag_i as r_i, 
p.modelMag_r - p.modelMag_z as r_z, 
p.petroRad_r,
p.flags, dbo.fPhotoFlagsN(p.flags) as flag_text
FROM #upload u
JOIN #x x ON x.up_id = u.up_id
JOIN PhotoObj p ON p.objID = x.objID 
ORDER BY x.up_id
'''


In [35]:
crossDf = CasJobs.executeQuery(query3, "dr16")
crossDf

,files,name,labels,reds,objID,type,ra,dec,modelMag_r,u_g,g_z,g_r,g_i,r_i,r_z,petroRad_r,flags,flag_text
0,sdss_ra=49.5885_dec=41.6896.png,PCC-2087,0,2.39,1237661059574204597,GALAXY,50,42,19,1,2,1,1,1,1,4,105624251793424,DEBLEND_NOPEAK DEBLENDED_AT_EDGE STATIONARY MO...
1,sdss_ra=50.004_dec=41.341.png,PCC-5423,0,5.62,1237661055281857482,GALAXY,50,41,18,2,2,1,2,1,1,4,35259802583056,DEBLENDED_AT_EDGE STATIONARY DEBLENDED_AS_MOVI...
2,sdss_ra=50.0018_dec=41.6806.png,PCC-5408,0,1.8725,1237661055818662907,STAR,50,42,18,1,1,1,1,0,0,1,35255507484688,DEBLENDED_AT_EDGE STATIONARY MOVED BINNED1 CHILD
3,sdss_ra=49.6743_dec=41.3287.png,PCC-2818,0,0.04,1237661122387969064,GALAXY,50,41,19,4,2,1,2,1,1,3,68987912448,STATIONARY BINNED1 NOPETRO
4,sdss_ra=49.4266_dec=41.3159.png,PCC-1064,0,1.8225,1237661083199145950,GALAXY,49,41,19,2,3,2,2,1,1,3,68988047360,STATIONARY BINNED1 INTERP COSMIC_RAY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,sdss_ra=49.9705_dec=41.6088.png,PCC-5169,1,6.5925,1237661059574334013,GALAXY,50,42,16,2,2,1,1,0,1,13,193583034536024,PSF_FLUX_INTERP DEBLENDED_AT_EDGE INTERP_CENTE...
203,sdss_ra=49.9947_dec=41.75.png,PCC-5358,1,49.0425,1237670960021504409,GALAXY,50,42,18,6,2,1,1,0,1,9,105622104310032,DEBLEND_NOPEAK DEBLENDED_AT_EDGE STATIONARY BI...
204,sdss_ra=49.9967_dec=41.3092.png,PCC-5374,1,17.355,1237661055281857031,STAR,50,41,23,1,2,1,1,-0,1,1,457465859019024,TOO_FEW_GOOD_DETECTIONS PSF_FLUX_INTERP DEBLEN...
205,sdss_ra=49.8825_dec=41.7447.png,PCC-4502,1,4.63,1237670458048446875,GALAXY,50,42,19,4,-1,1,1,1,-1,8,527834569642256,TOO_FEW_GOOD_DETECTIONS PSF_FLUX_INTERP DEBLEN...


In [36]:
pccFlags = crossDf['flag_text']
pccTypes = crossDf['type']

In [37]:
# pccTypes.unique()
crossDf['type'].value_counts()


GALAXY    197
STAR       10
Name: type, dtype: int64

In [38]:
pccStars = crossDf.loc[crossDf['type'] == 'STAR']
pccStars

,files,name,labels,reds,objID,type,ra,dec,modelMag_r,u_g,g_z,g_r,g_i,r_i,r_z,petroRad_r,flags,flag_text
2,sdss_ra=50.0018_dec=41.6806.png,PCC-5408,0,1.8725,1237661055818662907,STAR,50,42,18,1,1,1,1,0,0,1,35255507484688,DEBLENDED_AT_EDGE STATIONARY MOVED BINNED1 CHILD
8,sdss_ra=49.4784_dec=41.2846.png,PCC-1388,0,3.62,1237661083199144672,STAR,49,41,18,1,1,1,1,0,0,1,35253360001040,DEBLENDED_AT_EDGE STATIONARY BINNED1 CHILD
20,sdss_ra=49.5144_dec=41.6052.png,PCC-1623,0,17.38,1237661055281660277,STAR,50,42,19,3,3,1,3,1,2,1,105622104576528,DEBLEND_NOPEAK DEBLENDED_AT_EDGE STATIONARY BI...
32,sdss_ra=49.4898_dec=41.5315.png,PCC-1467,0,13.855,1237661055281660365,STAR,49,42,22,1,3,2,2,1,1,1,35253393690896,DEBLENDED_AT_EDGE STATIONARY BINNED1 DEBLENDED...
54,sdss_ra=49.623_dec=41.7693.png,PCC-2383,0,2.5125,1237661059574202849,STAR,50,42,19,2,1,1,1,0,0,1,193583068086288,PSF_FLUX_INTERP DEBLENDED_AT_EDGE INTERP_CENTE...
55,sdss_ra=49.9366_dec=41.7691.png,PCC-4910,0,2.935,1237670458048447673,STAR,50,42,19,1,1,0,1,0,0,1,68987912448,STATIONARY BINNED1 NOPETRO
60,sdss_ra=49.9842_dec=41.2138.png,PCC-5265,0,0.8025,1237661055281857614,STAR,50,41,18,2,1,1,1,0,0,1,144115257063768064,MAYBE_EGHOST STATIONARY BINNED1
62,sdss_ra=49.7175_dec=41.6788.png,PCC-3159,0,12.7875,1237661059574268000,STAR,50,42,20,3,3,2,3,1,2,1,72092813071876112,MAYBE_CR DEBLENDED_AT_EDGE BAD_MOVING_FIT BINN...
196,sdss_ra=49.9453_dec=41.3786.png,PCC-4981,1,9.1025,1237661055281857728,STAR,50,41,19,4,4,1,3,2,2,3,35255507615760,DEBLENDED_AT_EDGE STATIONARY MOVED BINNED1 INT...
204,sdss_ra=49.9967_dec=41.3092.png,PCC-5374,1,17.355,1237661055281857031,STAR,50,41,23,1,2,1,1,-0,1,1,457465859019024,TOO_FEW_GOOD_DETECTIONS PSF_FLUX_INTERP DEBLEN...


In [39]:
template = lambda starID : f'https://skyserver.sdss.org/dr16/en/tools/explore/Summary.aspx?id={starID}'
# jpg_temp = lambda starID : 'https://skyserver.sdss.org/dr16/SkyServerWS/ImgCutout/'\
#           f'getjpeg?TaskName=Skyserver.Explore.Image&id={starID}&scale=0.1&width=200&height=200'
for x in pccStars['objID']:
    print(template(x))
    print(x)
#     print(jpg_temp(x))


https://skyserver.sdss.org/dr16/en/tools/explore/Summary.aspx?id=1237661055818662907
1237661055818662907
https://skyserver.sdss.org/dr16/en/tools/explore/Summary.aspx?id=1237661083199144672
1237661083199144672
https://skyserver.sdss.org/dr16/en/tools/explore/Summary.aspx?id=1237661055281660277
1237661055281660277
https://skyserver.sdss.org/dr16/en/tools/explore/Summary.aspx?id=1237661055281660365
1237661055281660365
https://skyserver.sdss.org/dr16/en/tools/explore/Summary.aspx?id=1237661059574202849
1237661059574202849
https://skyserver.sdss.org/dr16/en/tools/explore/Summary.aspx?id=1237670458048447673
1237670458048447673
https://skyserver.sdss.org/dr16/en/tools/explore/Summary.aspx?id=1237661055281857614
1237661055281857614
https://skyserver.sdss.org/dr16/en/tools/explore/Summary.aspx?id=1237661059574268000
1237661059574268000
https://skyserver.sdss.org/dr16/en/tools/explore/Summary.aspx?id=1237661055281857728
1237661055281857728
https://skyserver.sdss.org/dr16/en/tools/explore/Summar

In [40]:
pccStars['flag_text'].to_list()

['DEBLENDED_AT_EDGE STATIONARY MOVED BINNED1 CHILD ',
 'DEBLENDED_AT_EDGE STATIONARY BINNED1 CHILD ',
 'DEBLEND_NOPEAK DEBLENDED_AT_EDGE STATIONARY BINNED1 SATURATED INTERP COSMIC_RAY MANYPETRO CHILD ',
 'DEBLENDED_AT_EDGE STATIONARY BINNED1 DEBLENDED_AS_PSF INTERP COSMIC_RAY NOPETRO CHILD ',
 'PSF_FLUX_INTERP DEBLENDED_AT_EDGE INTERP_CENTER STATIONARY BINNED1 DEBLENDED_AS_PSF INTERP CHILD ',
 'STATIONARY BINNED1 NOPETRO ',
 'MAYBE_EGHOST STATIONARY BINNED1 ',
 'MAYBE_CR DEBLENDED_AT_EDGE BAD_MOVING_FIT BINNED1 DEBLENDED_AS_PSF INTERP CHILD ',
 'DEBLENDED_AT_EDGE STATIONARY MOVED BINNED1 INTERP CHILD ',
 'TOO_FEW_GOOD_DETECTIONS PSF_FLUX_INTERP DEBLENDED_AT_EDGE STATIONARY BINNED1 DEBLENDED_AS_PSF SATURATED INTERP COSMIC_RAY NOPETRO CHILD ']

In [41]:
files = pccStars['files'].to_list()
files[0].split('=')

['sdss_ra', '50.0018_dec', '41.6806.png']

In [42]:
# ra = df0[2]
# dec = df0[3]

# # think about pausing every 100 images or so

# for i in range(len(ra)):
# # for i in range(800):
#     urlVar = f'https://skyserver.sdss.org/dr16/SkyServerWS/ImgCutout/'\
#     f'getjpeg?TaskName=Skyserver.Explore.Image&ra={str(ra[i]).strip()}'\
#     f'&dec={str(dec[i]).strip()}&scale=0.1&width=200&height=200'
    
#     # tell the loop to pause for a bit - every 100 images
#     if i%100==0:
#         time.sleep(5)
        
#     img_data = requests.get(urlVar).content
#     with open(f'SDSS1/sdss_ra={ra[i]}_dec={dec[i]}.png', 'wb') as handler:
#         handler.write(img_data)

KeyError: 2